Goal : Preserving conversational history and generating summary at the end of converstaion

Step 1: importing required packages

In [ ]:
! pip install  langgraph
! pip install langchain 
! pip install langchain_core_google_genai 
! pip install typing_extensions 
! pip install langchain_core


In [ ]:
from langgraph.graph import START, END, StateGraph, add_messages , MessagesState
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Literal, Annotated
from typing_extensions import TypedDict 
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage ,RemoveMessage,SystemMessage
from _collections_abc import Sequence


Step 2: Define State

In [ ]:
class State(MessagesState):
    summary:str

In [ ]:
test_state= State()

In [ ]:
test_state["summary"]

In [ ]:
bool(test_state.get("summary", ""))

Step 4: Define node

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0, seed  =365, max_tokens=100)


In [ ]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
    
    system_message=f"here's quick summary of what's been disscuss so far: {state.get('summary', '')} . keep this in mind as you answer the next question."
    response = chat.invoke([SystemMessage(system_message) + state["messages"]])
    response.pretty_print()
    
    return State(messages = [response])

In [ ]:
def ask_another_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_another_question:")
    
    question = "Would you like to ask one more question (yes/no)?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def summarize_and_delete_messages(state: State) -> State:
    print(f"\n-------> ENTERING trim_messages:")
    
    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
        
    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below.  
Build upon the previous summary rather than repeating it so that the result  
reflects the most recent context and developments.


Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''
    
    print(summary_instructions)
    
    summary = chat.invoke([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]
    
    return State(messages = remove_messages, summary = summary.content)

Step 5: Define routing function

In [ ]:
def routing_function(state: State) -> Literal["summarize_and_delete_messages", "__end__"]:
    
    if state["messages"][-1].content == "yes":
        return "summarize_and_delete_messages"
    else:
        return "__end__"

Step 6: define graph

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)
graph.add_node("summarize_and_delete_messages", summarize_and_delete_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)
graph.add_edge("summarize_and_delete_messages", "ask_question")

In [ ]:
graph_compiled = graph.compile()

In [ ]:
graph_compiled

Step 7: Test the graph

In [ ]:
graph_compiled.invoke(State(messages = []))